In [1]:
import sys
sys.path.append('../../')
import bemppUQ
import numpy as np
import bempp.api 
from bempp.api.assembly.blocked_operator import BlockedOperator
from bemppUQ.utils.mie import bhmie
from bemppUQ.preconditioning.osrc import osrc_MtE
from bemppUQ.utils.login import rescale, gmres
from bempp.api.assembly.blocked_operator import \
        coefficients_of_grid_function_list, \
        projections_of_grid_function_list, \
        grid_function_list_from_coefficients

In [6]:
#bempp.api.global_parameters.hmat.eps = 1E-8
bempp.api.global_parameters.quadrature.double_singular = 12
bempp.api.global_parameters.quadrature.far.double_order = 12
bempp.api.global_parameters.quadrature.near.double_order = 12

bempp.api.global_parameters.assembly.boundary_operator_assembly_type = 'dense'
m_ext = 1
m_int = 1

k_ext = 1.047197551196598
n = 1.311
k_int = n*k_ext

precision = 10
h = 2*np.pi/(precision*k_ext)

maxiter = 100

restart = 10000
tolerance =  1e-04

In [7]:
def incident_field_z(x):
    return np.array([0. * x[0], 0. * x[0], np.exp(1j * k_ext * x[0])])

# Incident field polarized in y-direction
def incident_field_y(x):
    return np.array([0. * x[0], np.exp(1j * k_ext * x[0]), 0. * x[0]])

# Dirichlet trace
def dirichlet_trace_z(x, n, domain_index, result):
    incident_field_z = np.array([0. * x[0], 0. * x[0], np.exp(1j * k_ext * x[0])])
    result[:] = np.cross(incident_field_z, n)

def dirichlet_trace_y(x, n, domain_index, result):
    incident_field_y = np.array([0. * x[0], np.exp(1j * k_ext * x[0]), 0. * x[0]])
    result[:] = np.cross(incident_field_y, n)

def neumann_trace_z(x, n, domain_index, result):
    curl_z = np.array([x[0] * 0. ,-1j * k_ext * np.exp(1j * k_ext * x[0]), x[0] * 0.])
#     with objmode():
    result[:] = (1/(1j * k_ext)) * np.cross(curl_z, n)

def neumann_trace_y(x, n, domain_index, result):
    curl_y = np.array([x[0] * 0., x[0] * 0. ,1j * k_ext * np.exp(1j * k_ext * x[0])])
#     with objmode():
    result[:] = (1/(1j * k_ext)) * np.cross(curl_y,n)

In [8]:
number_of_angles = 3601
angles = np.pi * np.linspace(0, 2, number_of_angles)
unit_points = np.array([np.cos(angles), np.sin(angles), np.zeros(number_of_angles)])

ka = k_ext
refIndex = n
numAngles = 901
[s1,s2,qext,qsca,qback,gsca]=bhmie(ka,refIndex,numAngles)

grid = bempp.api.shapes.sphere(h=h)
#grid = bemppUQ.shapes.reentrant_cube_exterior(h=0.1)
#grid = bempp.api.shapes.reentrant_cube(h=h)

RWG = bempp.api.function_space(grid, 'RWG', 0)
SNC = bempp.api.function_space(grid, 'SNC', 0)

number_of_angles = 3601
angles = np.pi * np.linspace(0, 2, number_of_angles)
unit_points = np.array([np.cos(angles), np.sin(angles), np.zeros(number_of_angles)])

grid = bempp.api.shapes.sphere(h=h)

RWG = bempp.api.function_space(grid, 'RWG', 0)

magnetic_far = bempp.api.operators.far_field.maxwell.magnetic_field(RWG, unit_points, k_ext)    
M = bempp.api.as_matrix(magnetic_far.discrete_operator)
print(M.max(), 'IS IT OK?')


print(RWG.global_dof_count * 2 , 'N')
op_int = BlockedOperator(2,2)
op_ext = BlockedOperator(2,2)
op_osrc_int = BlockedOperator(2,2)
op_osrc_ext = BlockedOperator(2,2)

efield_e = bempp.api.operators.boundary.maxwell.electric_field(RWG, RWG, SNC, k_ext)
mfield_e = bempp.api.operators.boundary.maxwell.magnetic_field(RWG, RWG, SNC, k_ext)

efield_i = bempp.api.operators.boundary.maxwell.electric_field(RWG, RWG, SNC, k_int)
mfield_i = bempp.api.operators.boundary.maxwell.magnetic_field(RWG, RWG, SNC, k_int)

osrc_ext = osrc_MtE(grid, k_ext)
osrc_int = osrc_MtE(grid, k_int)

op_ext[0,0] = mfield_e
op_ext[0,1] = efield_e
op_ext[1,0] = -1*efield_e
op_ext[1,1] = mfield_e

op_int[0,0] = mfield_i
op_int[0,1] = efield_i
op_int[1,0] = -1*efield_i
op_int[1,1] = mfield_i

op_osrc_ext[0,1] = osrc_ext
op_osrc_ext[1,0] = -osrc_ext

op_osrc_int[0,1] = osrc_int
op_osrc_int[1,0] = -osrc_int

op_osrc_ext = rescale(op_osrc_ext, k_ext, m_ext)
op_osrc_int = rescale(op_osrc_int, k_int, m_int)

op_osrc = op_osrc_ext + op_osrc_int

op_ext = rescale(op_ext, k_ext, m_ext)
op_int = rescale(op_int, k_int, m_int)

op = op_ext + op_int

identity = BlockedOperator(2,2)
identity[0, 0] = bempp.api.operators.boundary.sparse.identity(RWG, RWG, SNC)
identity[1, 1] = bempp.api.operators.boundary.sparse.identity(RWG, RWG, SNC)

#Discretize the incident field
incident_dirichlet_z = bempp.api.GridFunction(RWG, fun=dirichlet_trace_z)
incident_neumann_z = (k_ext/m_ext) * bempp.api.GridFunction(RWG, fun=neumann_trace_z)

incident_dirichlet_y = bempp.api.GridFunction(RWG, dual_space=RWG, fun=dirichlet_trace_y)
incident_neumann_y = (k_ext/m_ext) * bempp.api.GridFunction(RWG, fun=neumann_trace_y)

op_rhs = 0.5 * identity - op_int

rhs_z = [incident_dirichlet_z, incident_neumann_z]
rhs_z = op_rhs * rhs_z

rhs_y = [incident_dirichlet_y, incident_neumann_y]
rhs_y = op_rhs * rhs_y

b_y = projections_of_grid_function_list(rhs_y, op.dual_to_range_spaces)
b_z = projections_of_grid_function_list(rhs_z, op.dual_to_range_spaces)

op_osrc_wf = op_osrc.weak_form()
op_wf = op.weak_form()

(0.0117601384696+0.00908157053242j) IS IT OK?
336 N


/usr/lib/python3/dist-packages/bempp/api/assembly/discrete_boundary_operator.py:340: RuntimeWarning: invalid value encountered in multiply
  return DenseDiscreteBoundaryOperator(self.A * other)


In [9]:
x_y, info, res_y, times_y = gmres(op_osrc_wf * op_wf ,op_osrc_wf * b_y, tol=tolerance, return_residuals=True,  maxiter=maxiter, restart=restart)
x_z, info, res_z, times_z = gmres(op_osrc_wf * op_wf ,op_osrc_wf * b_z, tol=tolerance, return_residuals=True,  maxiter=maxiter, restart=restart)

sol_y = grid_function_list_from_coefficients(x_y, op.domain_spaces)
sol_z = grid_function_list_from_coefficients(x_z, op.domain_spaces)

iteration - 1 || residual - nan 0.013988256454467773
iteration - 2 || residual - nan 0.0031948089599609375
iteration - 3 || residual - nan 0.002635955810546875
iteration - 4 || residual - nan 0.002618074417114258
iteration - 5 || residual - nan 0.002649068832397461
iteration - 6 || residual - nan 0.002707242965698242
iteration - 7 || residual - nan 0.002701997756958008
iteration - 8 || residual - nan 0.0027158260345458984
iteration - 9 || residual - nan 0.0027358531951904297
iteration - 10 || residual - nan 0.004095554351806641
iteration - 11 || residual - nan 0.0028638839721679688
iteration - 12 || residual - nan 0.0027959346771240234
iteration - 13 || residual - nan 0.002790689468383789
iteration - 14 || residual - nan 0.0028238296508789062
iteration - 15 || residual - nan 0.0028345584869384766
iteration - 16 || residual - nan 0.0028829574584960938
iteration - 17 || residual - nan 0.002880573272705078
iteration - 18 || residual - nan 0.0028994083404541016
iteration - 19 || residual -

iteration - 54 || residual - nan 0.0039441585540771484
iteration - 55 || residual - nan 0.0035648345947265625
iteration - 56 || residual - nan 0.003551959991455078
iteration - 57 || residual - nan 0.0035676956176757812
iteration - 58 || residual - nan 0.003658294677734375
iteration - 59 || residual - nan 0.0036132335662841797
iteration - 60 || residual - nan 0.0036973953247070312
iteration - 61 || residual - nan 0.0037877559661865234
iteration - 62 || residual - nan 0.003651142120361328
iteration - 63 || residual - nan 0.003686666488647461
iteration - 64 || residual - nan 0.003781557083129883
iteration - 65 || residual - nan 0.0038406848907470703
iteration - 66 || residual - nan 0.0037169456481933594
iteration - 67 || residual - nan 0.0037920475006103516
iteration - 68 || residual - nan 0.003773927688598633
iteration - 69 || residual - nan 0.0038106441497802734
iteration - 70 || residual - nan 0.0038056373596191406
iteration - 71 || residual - nan 0.0038137435913085938
iteration - 72 |

In [10]:
far_field_y = np.zeros((3, number_of_angles), dtype='complex128')
far_field_z = np.zeros((3, number_of_angles), dtype='complex128')
# bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'

electric_far = bempp.api.operators.far_field.maxwell.electric_field(RWG, unit_points, k_ext)
magnetic_far = bempp.api.operators.far_field.maxwell.magnetic_field(RWG, unit_points, k_ext)    
M = bempp.api.as_matrix(magnetic_far.discrete_operator)
print(M.max(), 'IS IT OK?')

far_field_y += -electric_far * (m_ext/k_ext *sol_y[1]) - magnetic_far * sol_y[0]
far_field_z += -electric_far * (m_ext/k_ext *sol_z[1]) - magnetic_far * sol_z[0]
    
#far_field_z

A22 = -1j * k_ext * far_field_z[2,:]   # -1j * kExt is to make agree with eqn 3.21 p70 Bohren and Huffman
A12 = -1j * k_ext * (- np.sin(angles) * far_field_z[0,:] + np.cos(angles) * far_field_z[1,:])

A11 = -1j * k_ext * (- np.sin(angles) * far_field_y[0,:] + np.cos(angles) * far_field_y[1,:])
A21 = -1j * k_ext * far_field_y[2,:]

(nan+nan*j) IS IT OK?


In [11]:
bempp.api.global_parameters.assembly.boundary_operator_assembly_type

'dense'

In [12]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['figure.figsize'] = (15, 4) # Increase the figure size in the notebook
params = {'legend.fontsize': 'x-large', 'axes.labelsize': 'x-large', 'axes.titlesize': 'x-large',
         'xtick.labelsize': 'x-large', 'ytick.labelsize': 'x-large'}
plt.rcParams.update(params)

fig, axes = plt.subplots(1, 2, sharex=True, sharey=False)

f0 = axes[0].semilogy(angles[:1801],abs(s2)**2,'-r',angles[:1801],abs(A11[:1801])**2,'--b')
axes[0].set_title('$A_{11}$: Mie (red) vs. BEM (blue)')

f1 = axes[1].semilogy(angles[:1801],abs(s1)**2,'-r',angles[:1801],abs(A22[:1801])**2,'--b')
axes[1].set_title('$A_{22}$: Mie (red) vs. BEM (blue)')


plt.show()

In [13]:
print(bempp.api.as_matrix(electric_far.discrete_operator).max())
print(bempp.api.as_matrix(magnetic_far.discrete_operator).max())

(0.0117598180161-0.00860349487895j)
(nan+nan*j)


In [14]:
electric = bempp.api.operators.potential.maxwell.electric_field(RWG, 4 *unit_points, k_ext)
magnetic = bempp.api.operators.potential.maxwell.magnetic_field(RWG, 4 *unit_points, k_ext)    


In [15]:
bempp.api.as_matrix(electric.discrete_operator).max()

(0.0031128804540334612-0.00070219029438008649j)

In [ ]:
bempp.api.as_matrix(magnetic.discrete_operator).max()